In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Data Loading
df = pd.read_csv('green_supply_chain_dataset_1000.csv')

In [3]:
# Initialize the MinMaxScaler
scaler = MinMaxScaler()

In [4]:
#Normalization
# Columns to normalize (Cost, CO2 Emissions, Sustainability Score)
columns_to_normalize = ['Cost_$', 'CO2_Emissions_kg', 'Waste_Generated_kg', 'Energy_Consumption_kWh', 'Sustainability_Score']

# Create a copy of the dataframe for normalized values
df_normalized = df.copy()

# Apply Min-Max scaling to the selected columns
df_normalized[columns_to_normalize] = scaler.fit_transform(df_normalized[columns_to_normalize])

# For objectives that are maximized (Sustainability_Score), transform them to be minimized
# (1 - normalized_value) so that a higher original score translates to a lower 'min' score
df_normalized['Sustainability_Score_min'] = 1 - df_normalized['Sustainability_Score']

print("Normalized Data Sample")
print(df_normalized[['Cost_$', 'CO2_Emissions_kg', 'Waste_Generated_kg', 'Energy_Consumption_kWh', 'Sustainability_Score']].head())



Normalized Data Sample
     Cost_$  CO2_Emissions_kg  Waste_Generated_kg  Energy_Consumption_kWh  \
0  0.940589          0.163604            0.073103                0.717471   
1  0.164980          0.783466            0.622614                0.072078   
2  0.234329          0.121748            0.981612                0.071249   
3  0.264349          0.634975            0.190182                0.011995   
4  0.037737          0.123099            0.792943                0.958079   

   Sustainability_Score  
0              0.869427  
1              0.314366  
2              0.360345  
3              0.558745  
4              0.412422  


In [26]:
# 1. Aggregated Method (Corrected with Weights)

weights = {
    'Cost_$': 0.30,
    'CO2_Emissions_kg': 0.30,
    'Waste_Generated_kg': 0.13,
    'Energy_Consumption_kWh': 0.13,
    'Sustainability_Score_min': 0.14 # Assuming transformed for minimization
}

# Calculate aggregated score in normalized dataframe
df_normalized['Aggregated_Score'] = (
    weights['Cost_$'] * df_normalized['Cost_$'] +
    weights['CO2_Emissions_kg'] * df_normalized['CO2_Emissions_kg'] +
    weights['Waste_Generated_kg'] * df_normalized['Waste_Generated_kg'] +
    weights['Energy_Consumption_kWh'] * df_normalized['Energy_Consumption_kWh'] +
    weights['Sustainability_Score_min'] * df_normalized['Sustainability_Score_min']
)

# Find index of the best solution
best_idx = df_normalized['Aggregated_Score'].idxmin()

# Extract raw data row corresponding to best solution
best_raw = df.loc[best_idx]

# Extract normalized data row for the best solution
best_norm = df_normalized.loc[best_idx]

# Display combined results
print("Aggregated Method Solution")
print("Raw Values:")
print(best_raw[['ID', 'Product_Type', 'Cost_$', 'CO2_Emissions_kg', 'Waste_Generated_kg', 'Energy_Consumption_kWh', 'Sustainability_Score']])
print("\nNormalized Values:")
print(best_norm[['Cost_$', 'CO2_Emissions_kg', 'Waste_Generated_kg', 'Energy_Consumption_kWh', 'Sustainability_Score_min', 'Aggregated_Score']])


Aggregated Method Solution
Raw Values:
ID                                730
Product_Type              Electronics
Cost_$                     308.020772
CO2_Emissions_kg            82.658015
Waste_Generated_kg           8.216503
Energy_Consumption_kWh     175.441943
Sustainability_Score            80.06
Name: 729, dtype: object

Normalized Values:
Cost_$                      0.022356
CO2_Emissions_kg            0.033487
Waste_Generated_kg          0.147331
Energy_Consumption_kWh      0.279124
Sustainability_Score_min    0.145688
Aggregated_Score            0.092588
Name: 729, dtype: object


In [24]:
# 2. Weighted Criteria (Minimax) Method

weights = {
    'Cost_$': 0.30,
    'CO2_Emissions_kg': 0.30,
    'Waste_Generated_kg': 0.13,
    'Energy_Consumption_kWh': 0.13,
    'Sustainability_Score_min': 0.14
}

reference_point = {
    'Cost_$': df_normalized['Cost_$'].min(),
    'CO2_Emissions_kg': df_normalized['CO2_Emissions_kg'].min(),
    'Waste_Generated_kg': df_normalized['Waste_Generated_kg'].min(),
    'Energy_Consumption_kWh': df_normalized['Energy_Consumption_kWh'].min(),
    'Sustainability_Score_min': df_normalized['Sustainability_Score_min'].min()
}

def compute_max_deviation(row):
    deviations = [
        weights[obj] * abs(row[obj] - reference_point[obj])
        for obj in weights
    ]
    return max(deviations)

df_normalized['Max_Weighted_Deviation'] = df_normalized.apply(compute_max_deviation, axis=1)

# Find index of best solution (minimum of max weighted deviation)
best_idx = df_normalized['Max_Weighted_Deviation'].idxmin()

# Get raw and normalized rows for best solution
best_raw = df.loc[best_idx]
best_norm = df_normalized.loc[best_idx]

# Display combined results
print(" Weighted Criteria Method")
print("Raw Values:")
print(best_raw[['ID', 'Product_Type', 'Cost_$', 'CO2_Emissions_kg', 'Waste_Generated_kg',
                'Energy_Consumption_kWh', 'Sustainability_Score']])
print("\nNormalized Values:")
print(best_norm[['Cost_$', 'CO2_Emissions_kg', 'Waste_Generated_kg', 'Energy_Consumption_kWh',
                 'Sustainability_Score_min', 'Max_Weighted_Deviation']])


 Weighted Criteria Method
Raw Values:
ID                                    89
Product_Type              Pharmaceutical
Cost_$                        702.371043
CO2_Emissions_kg              154.369958
Waste_Generated_kg              4.381372
Energy_Consumption_kWh        122.032236
Sustainability_Score               90.22
Name: 88, dtype: object

Normalized Values:
Cost_$                      0.104861
CO2_Emissions_kg            0.109093
Waste_Generated_kg          0.069027
Energy_Consumption_kWh      0.160223
Sustainability_Score_min    0.016639
Max_Weighted_Deviation      0.032728
Name: 88, dtype: object


In [28]:
# Episilon- Constraint method
# Step 1: Define Epsilon Thresholds (adjustable)
epsilon_waste = df['Waste_Generated_kg'].quantile(0.2)
epsilon_energy = df['Energy_Consumption_kWh'].quantile(0.2)
epsilon_sustainability = df['Sustainability_Score'].quantile(0.8)

# Step 2: Filter data using epsilon constraints
epsilon_filtered_df = df[
    (df['Waste_Generated_kg'] <= epsilon_waste) &
    (df['Energy_Consumption_kWh'] <= epsilon_energy) &
    (df['Sustainability_Score'] >= epsilon_sustainability)
]

# Step 3: Among filtered data, find those with minimum Cost
if not epsilon_filtered_df.empty:
    min_cost = epsilon_filtered_df['Cost_$'].min()
    min_cost_df = epsilon_filtered_df[epsilon_filtered_df['Cost_$'] == min_cost]

    # Step 4: Among those, choose the one with minimum CO2
    final_solution = min_cost_df.loc[min_cost_df['CO2_Emissions_kg'].idxmin()]

    # Step 5: Display the solution
    print("\nTwo-Objective Epsilon-Constraint Method (Minimize Cost and CO2 emissions)")
    print(final_solution[[
        'ID', 'Product_Type', 'Cost_$', 'CO2_Emissions_kg',
        'Waste_Generated_kg', 'Energy_Consumption_kWh', 'Sustainability_Score'
    ]])
else:
    print("\nNo solution found for the Epsilon-Constraint Method with the specified thresholds.")
    print("Consider relaxing constraints on Waste, Energy, or Sustainability.")



Two-Objective Epsilon-Constraint Method (Minimize Cost and CO2 emissions)
ID                                    89
Product_Type              Pharmaceutical
Cost_$                        702.371043
CO2_Emissions_kg              154.369958
Waste_Generated_kg              4.381372
Energy_Consumption_kWh        122.032236
Sustainability_Score               90.22
Name: 88, dtype: object


In [22]:
# Lexicographic Method

# Step 1: Create composite score for Cost + CO2 (equal weights; adjust if needed)
df['Primary_Score'] = df['Norm_Cost'] + df['Norm_CO2']

# Step 2: Minimize Primary Score (Cost + CO2)
min_primary_score = df['Primary_Score'].min()
lex_df_1 = df[df['Primary_Score'] == min_primary_score]

# Step 3: Among those, minimize Waste
min_waste = lex_df_1['Waste_Generated_kg'].min()
lex_df_2 = lex_df_1[lex_df_1['Waste_Generated_kg'] == min_waste]

# Step 4: Among those, minimize Energy Consumption
min_energy = lex_df_2['Energy_Consumption_kWh'].min()
lex_df_3 = lex_df_2[lex_df_2['Energy_Consumption_kWh'] == min_energy]

# Step 5: Among those, maximize Sustainability Score
max_sustainability = lex_df_3['Sustainability_Score'].max()
final_lex_solution = lex_df_3[lex_df_3['Sustainability_Score'] == max_sustainability].iloc[0]

# Print the result
print("Lexicographic Method with Joint Cost + CO2 Minimization")
print(final_lex_solution[[
    'ID', 'Product_Type', 'Cost_$', 'CO2_Emissions_kg',
    'Waste_Generated_kg', 'Energy_Consumption_kWh', 'Sustainability_Score'
]])


Lexicographic Method with Joint Cost + CO2 Minimization (Using Normalized Data)
ID                                   270
Product_Type              Pharmaceutical
Cost_$                        271.309388
CO2_Emissions_kg               84.877296
Waste_Generated_kg             14.102699
Energy_Consumption_kWh         403.82519
Sustainability_Score                77.3
Name: 269, dtype: object


In [29]:
#  Preemptive Goal Programming
# Compute positive deviations from goals
df['Deviation_Cost'] = (df['Cost_$'] - goal_cost).clip(lower=0)
df['Deviation_CO2'] = (df['CO2_Emissions_kg'] - goal_co2).clip(lower=0)
df['Deviation_Waste'] = (df['Waste_Generated_kg'] - goal_waste).clip(lower=0)
df['Deviation_Energy'] = (df['Energy_Consumption_kWh'] - goal_energy).clip(lower=0)
df['Deviation_Sustainability'] = (goal_sustainability - df['Sustainability_Score']).clip(lower=0)

# Combine Cost and CO2 deviations as Priority 1 as both cost and co2 emmission hold important
df['Priority1_Deviation'] = df['Deviation_Cost'] + df['Deviation_CO2']

# Priority 1: Minimize combined Cost and CO2 deviation
min_dev_priority1 = df['Priority1_Deviation'].min()
df_stage1 = df[df['Priority1_Deviation'] == min_dev_priority1]

# Priority 2: Minimize Waste deviation among above
min_dev_waste = df_stage1['Deviation_Waste'].min()
df_stage2 = df_stage1[df_stage1['Deviation_Waste'] == min_dev_waste]

# Priority 3: Minimize Energy deviation
min_dev_energy = df_stage2['Deviation_Energy'].min()
df_stage3 = df_stage2[df_stage2['Deviation_Energy'] == min_dev_energy]

# Priority 4: Minimize Sustainability deviation (maximize sustainability)
min_dev_sustain = df_stage3['Deviation_Sustainability'].min()
final_pg_solution = df_stage3[df_stage3['Deviation_Sustainability'] == min_dev_sustain].iloc[0]

# Display Final Solution
print("\nPreemptive Goal Programming Solution with Combined Cost & CO2 Priority")
print(final_pg_solution[[
    'ID', 'Product_Type', 'Cost_$', 'CO2_Emissions_kg', 'Waste_Generated_kg',
    'Energy_Consumption_kWh', 'Sustainability_Score',
    'Deviation_Cost', 'Deviation_CO2', 'Deviation_Waste', 'Deviation_Energy', 'Deviation_Sustainability'
]])



Preemptive Goal Programming Solution with Combined Cost & CO2 Priority
ID                                     270
Product_Type                Pharmaceutical
Cost_$                          271.309388
CO2_Emissions_kg                 84.877296
Waste_Generated_kg               14.102699
Energy_Consumption_kWh           403.82519
Sustainability_Score                  77.3
Deviation_Cost                   70.143303
Deviation_CO2                    33.981052
Deviation_Waste                  13.102129
Deviation_Energy                353.764578
Deviation_Sustainability             14.23
Name: 269, dtype: object
